In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import bson
import io
from PIL import Image
import struct
import pickle
import tensorflow as tf

In [ ]:
'''train_file = "/kaggle/input/cdiscount-image-classification-challenge/train.bson"
IDS_mapping = {}
offset = 0
length_size = 4
i = 0
cat = {}
p = 0
with open(train_file , "rb") as f:
    while True:
        f.seek(offset)
        item_length_bytes = f.read(length_size) 
        if len(item_length_bytes) == 0:
            break
        length = struct.unpack("<i", item_length_bytes)[0]
        f.seek(offset)
        item_data = f.read(length)
        item = bson.BSON.decode(item_data)
        ca = int(item["category_id"])
        if ca not in cat.keys():
            cat[ca] = p
            p += 1
        for j in range(len(item["imgs"])):
            IDS_mapping[i + j] = (offset , length , j , cat[ca])
        i += len(item["imgs"])
        offset += length'''

In [ ]:
a_file = open("../input/till-saving-mapping/mappings.pkl" , "rb")
IDS_mapping = pickle.load(a_file)

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a floast_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
  array = tf.io.serialize_tensor(array)
  return array

In [ ]:
def for_single_image(ind , image , label):
  real_data = {
      'index' : _int64_feature(ind) , 
      'raw_image' : _bytes_feature(serialize_array(image)) , 
      'label' : _int64_feature(label)
  }
  out = tf.train.Example(features=tf.train.Features(feature=real_data))
  return out

In [ ]:
len(IDS_mapping.keys())

In [ ]:
save_name = "try_tfrec.tfrecords"
writer = tf.io.TFRecordWriter(save_name)
#keys = IDS_mapping.keys()
#values = IDS_mapping.values()
f = open("/kaggle/input/cdiscount-image-classification-challenge/train.bson" , "rb")
for i in range(0 , int(len(IDS_mapping.keys())/4)):
  ind = i
  offset , length , j , cat = IDS_mapping[ind]
  f.seek(offset)
  item = f.read(length)
  data = bson.BSON.decode(item)
  img = Image.open(io.BytesIO(data["imgs"][j]["picture"]))
  out = for_single_image(ind , np.array(img) , cat)
  writer.write(out.SerializeToString())
writer.close()